# 분류모델 평가

## 01.이진 분류 평가


<table border="1" cellpadding="5" cellspacing="0">
  <thead>
    <tr>
      <th>이름</th>
      <th>설명</th>
      <th>관련 수식</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Accuracy (정확도)</td>
      <td>전체 샘플 중에서 올바르게 예측된 샘플의 비율을 나타낸다. 하지만 데이터가 불균형할 경우 유용하지 않을 수 있다.</td>
      <td>$$
      \text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}
      $$</td>
    </tr>
    <tr>
      <td>Precision (정밀도)</td>
      <td>모델이 Positive로 예측한 샘플 중 실제로 Positive인 샘플의 비율을 나타낸다. FP를 줄이는 데 중점을 둔다.</td>
      <td>$Precision = \frac{TP}{TP + FP}$</td>
    </tr>
    <tr>
      <td>Recall (재현율 또는 민감도)</td>
      <td>실제 Positive 샘플 중 모델이 Positive로 정확히 예측한 샘플의 비율을 나타낸다. FN을 줄이는 데 중점을 둔다.</td>
      <td>$Recall = \frac{TP}{TP + FN}$</td>
    </tr>
    <tr>
      <td>F1-Score (F1 점수)</td>
      <td>Precision과 Recall의 조화 평균을 나타내며, 불균형 데이터에서도 유용하다.</td>
      <td>$F1 = 2 \cdot \frac{Precision \cdot Recall}{Precision + Recall}$</td>
    </tr>
    <tr>
      <td>특이도 (Specificity)</td>
      <td>실제 부정인 것 중에서 모델이 부정으로 정확하게 예측한 비율</td>
      <td>$Specificity=\frac{TN}{TN + FP}$</td>
    </tr>
    <tr>
      <td>ROC Curve (수신자 조작 특성 곡선)</td>
      <td>모델의 분류 기준을 변경했을 때, True Positive Rate (Recall)와 False Positive Rate 간의 관계를 시각화한 그래프이다.</td>
      <td>$FPR = \frac{FP}{FP + TN},\ TPR = \frac{TP}{TP + FN}$</td>
    </tr>
    <tr>
      <td>AUC (Area Under Curve)</td>
      <td>ROC Curve 아래 면적을 나타내며, 1에 가까울수록 좋은 성능을 의미한다.</td>
      <td>AUC는 ROC Curve 아래의 면적을 의미</td>
    </tr>
  </tbody>
</table>


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# 타이타닉 데이터 로드

titanic_df = pd.read_csv('data/titanic_train.csv')
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
# 결측치 체크
titanic_df.info() # Age, Cabin, Embarked 결측치 있음
titanic_df[['Cabin', 'Embarked']]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,Cabin,Embarked
0,NaN,S
1,C85,C
2,NaN,S
3,C123,S
4,NaN,S
...,...,...
886,NaN,S
887,B42,S
888,NaN,S
889,C148,C


In [23]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

# 전처리 함수
def fillna(df):
    # 결측치 처리 함수
    # df['Age'] = df['Age'].fillna(df['Age'].mean()) # 평균값으로 대체
    num_imputer = SimpleImputer(strategy='mean') # 평균으로 채워라 / SimpleImputer에 커서 올리면 strategy 있음
    df[['Age']] = num_imputer.fit_transform(df[['Age']]) # 데이터프레임 형태와 차원을 유지시키기 위해 대괄호 사용해주어야함

    # 기본값을 지정해주고 싶을 경우 strategy='constant'
    # => mean은 숫자라서 가능한데, 기본값을 바꾸고 싶으면 constant
    # 이 때, 채워넣을 값 : fill_value
    cat_imputer = SimpleImputer(strategy='constant', fill_value='N')
    df[['Cabin']] = cat_imputer.fit_transform(df[['Cabin']])



    # 가장 자주 나오는 값으로 대체 : most_frequent
    freq_imputer = SimpleImputer(strategy = 'most_frequent')
    df[['Embarked']] = freq_imputer.fit_transform(df[['Embarked']])

    return df
    pass

def drop_features(df):
    # 머신러닝 학습에 필요없는 컬럼 제거 함수
    return df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
    pass


sex_encoder = LabelEncoder()
cabin_encoder = LabelEncoder()
embarked_encoder = LabelEncoder()

def format_features(df):
    # 인코딩 또는 데이터 포멧을 조정 함수
    # -> 머신러닝에 전달되는 모든 특성은 숫자형이어야 한다.

    # Cabin은 선실영역값을 대체해서 사용
    df["Cabin"] = df['Cabin'].str[:1] # 첫번째 문자만 가져오기
    features = ['Sex', 'Cabin', 'Embarked']
    encoders = [sex_encoder, cabin_encoder, embarked_encoder]

    for i, feat in enumerate(features):
        encoder = encoders[i]
        df[feat] = encoder.fit_transform(df[feat])

    return df

def preprocess_titanic(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)

    return df

In [24]:
# 타이타닉 데이터 전처리
# 입력/라벨 분할
y_titanic = titanic_df['Survived']
X_titanic = titanic_df.drop(['Survived'], axis=1)

X_titanic = preprocess_titanic(X_titanic)

X_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    int64  
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Cabin     891 non-null    int64  
 7   Embarked  891 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 55.8 KB


In [25]:
# 학습/테스트 데이터 부리
from sklearn.model_selection import train_test_split
                                                                        # 8:2 비율
X_train, X_test, y_train, y_test = train_test_split(X_titanic, y_titanic, test_size=0.2, random_state=42,
                                                    stratify=y_titanic)

print(X_train.shape, X_test.shape) # 2차원이면 준비 완!
print(y_train.shape, y_test.shape)

(712, 8) (179, 8)
(712,) (179,)


## 정확도 Accuracy
모든 샘플 수 중의 정답 비율 $
Accuracy = \frac{TP + TN}{TP + TN + FP + FN}
$


데이터가 불균형한 경우(즉, 긍정과 부정 샘플의 수가 크게 다른 경우) 정확도는 비현실적인 성능을 나타낼 수 있다.


불균형 데이터 예시:
만약 실제로 1000개의 데이터 중에서 990개가 부정(Negative)이고 10개만이 긍정(Positive)인 경우라면, 모든 샘플을 부정으로만 예측해도 정확도는 99%입니다.


$ \text { Accuracy }=\frac{990}{1000}=99 \% $


In [62]:
# 모델 학습/평가
from sklearn.linear_model import LogisticRegression # 분류 모델

lr_reg = LogisticRegression(max_iter=10000)
lr_reg.fit(X_train, y_train)

print('학습셋 : ', lr_reg.score(X_train, y_train))
print('평가셋 : ', lr_reg.score(X_test, y_test))

학습셋 :  0.7949438202247191
평가셋 :  0.8156424581005587


C:\Users\TJ\anaconda3\envs\ai_env2\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
cabin_encoder.classes_
sex_encoder.classes_

array(['female', 'male'], dtype=object)

In [40]:
# 성별 생존여부 파악
titanic_df.groupby('Sex')['Survived'].value_counts()

Sex     Survived
female  1           233
        0            81
male    0           468
        1           109
Name: count, dtype: int64

In [42]:
# 성별만으로 생존여부를 판단하는 모델
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score

class MyTitanicCalssifier(BaseEstimator):
    def fit(self, X, y):
        # 아무것도 학습하지 않도록 오버라이드
        pass
    def predict(self, X):
        # 입력데이터의 성별컬럼(Sex)의 값에 따라 생존여부 예측
        # - 여성(0)인 경우 : 1 생존 예측
        # - 남성(1)인 경우 : 0 사망예측

        pred = np.zeros((X.shape[0], 1))
        # 모든 데이터를 순회하며 성별 검사
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i] == 0: # 여성인 경우
                    # 위치기반인덱싱
                    pred[i] = 1 # 생존
        return pred

    def score(self, X, y):
        # 정확도 측정
        return accuracy_score(y, self.predict(X))


my_clf = MyTitanicCalssifier()
my_clf.fit(X_train, y_train)

# 평가
print('학습셋 : ', my_clf.score(X_train, y_train))
print('평가셋 : ', my_clf.score(X_test, y_test))


학습셋 :  0.7893258426966292
평가셋 :  0.776536312849162


### 혼동행렬 Confusion Matrix


|               | 예측 값 부정 (Negative) | 예측 값 긍정 (Positive) |
|---------------|------------------------|------------------------|
| 실제 값 부정 (Negative) | True Negative (TN)         | False Positive (FP)        |
| 실제 값 긍정 (Positive) | False Negative (FN)        | True Positive (TP)         |


![](https://d.pr/i/2c98lS+)

In [44]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, my_clf.predict(X_test))

# 정답 / 전체샘플 수
(94 + 45)/ (94 + 16 + 24 + 45)

0.776536312849162

In [47]:
# 불균형 데이터 : 사망자 수가 생존자보다 많다.
titanic_df['Survived'].value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

In [48]:
# 무조건 사망으로 예측 모델



# 성별만으로 생존여부를 판단하는 모델
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score

class MyTitanicCalssifier2(BaseEstimator):
    def fit(self, X, y):
        # 아무것도 학습하지 않도록 오버라이드
        pass
    def predict(self, X):
        pred = np.zeros((X.shape[0], 1))
        return pred

    def score(self, X, y):
        # 정확도 측정
        return accuracy_score(y, self.predict(X))


my_clf = MyTitanicCalssifier2()
my_clf.fit(X_train, y_train)

# 평가
print('학습셋 : ', my_clf.score(X_train, y_train))
print('평가셋 : ', my_clf.score(X_test, y_test))


학습셋 :  0.6165730337078652
평가셋 :  0.6145251396648045


In [49]:
# 혼동행렬 확인
confusion_matrix(y_test, my_clf.predict(X_test))

array([[110,   0],
       [ 69,   0]])

In [54]:
# LogisticRegression 혼동행렬 확인
print(confusion_matrix(y_test, lr_reg.predict(X_test)))
# print('Accuracy : ', lr_reg.score(X_test, y_test))
print("Accuracy :", accuracy_score(y_test, lr_reg.predict(X_test)))

[[96 14]
 [21 48]]
Accuracy : 0.8044692737430168


### Precision 정밀도


$정밀도 = \frac{TP}{TP + FP}$


Positive(양성)이라고 예측한 것중에 정답인 확률


- 양성이라고 한것 중에 진짜 정답은 90%야~
- 스팸메일 분류한 것들중은 모두 스팸메일이던데~


**정밀도가 중요한 지표**
음성이 데이터를 양성으로 예측하면 큰일나는 경우
- 스팸메일 분류 (스팸메일이 아닌데, 스팸으로 분류하면 업무상 큰 혼란을 야기한다. 반면, 양성을 음성으로 분류하는 것(스팸메일을 분류하지 못한것)은 상대적으로 피해가 적다.)




- 암이 아닌 사람을 암으로 분류 했을때 큰 문제일까? -> 문제 x 왜냐하면 암이 아니기때문에 죽지 않는다.

In [50]:
from sklearn.metrics import precision_score

# 모두 사망으로 예측해서 정확도 61% 획득한 모델 -> 정밀도 0점
precision_score(y_test, my_clf.predict(X_test))



C:\Users\TJ\anaconda3\envs\ai_env2\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


0.0

In [55]:
# LogisticRegression 정밀도
precision_score(y_test, lr_reg.predict(X_test))

0.7741935483870968

### Recall 재현율
$재현율 = \frac{TP}{FN + TP}$


실제 Positive(양성)인 대상중에 Positive라고 예측한 확률


- 실제 암환자중에서는 70%를 맞췄네~




**재현율이 중요한 지표**
양성인 데이터를 음성으로 잘못 판단하면 큰일나는 경우.
- 보험/금융사기
- 암진단


In [51]:
from sklearn.metrics import recall_score

recall_score(y_test, my_clf.predict(X_test))

0.0

In [59]:
# LogisticRegression 재현율
recall_score(y_test, lr_reg.predict(X_test))
# 실제 생존 중에는 69%를 맞췄다.

0.6956521739130435

In [71]:
# 이진분류 종합평가함수
def evaluate_binary_classification(y_true, y_pred):
    # y_true : 실제값
    # y_pred : 예측값

    # "{0: .2f}".format(값) : 0번 인자 값을 소수점 둘째 자리까지 표현
    print('혼동행렬: \n', confusion_matrix(y_true, y_pred))
    print("정확도(Accuracy) : {0: .2f}".format(accuracy_score(y_true, y_pred)))
    print("정밀도(Precision) : {0: .2f}".format(precision_score(y_true, y_pred)))
    print("재현율(Recall) : {0: .2f}".format(recall_score(y_true, y_pred)))


evaluate_binary_classification(y_test, lr_reg.predict(X_test))

혼동행렬: 
 [[97 13]
 [20 49]]
정확도(Accuracy) :  0.82
정밀도(Precision) :  0.79
재현율(Recall) :  0.71


In [63]:
evaluate_binary_classification(y_test, my_clf.predict(X_test))

혼동행렬: 
 [[110   0]
 [ 69   0]]
정확도(Accuracy) :  0.61
정밀도(Precision) :  0.00
재현율(Recall) :  0.00


C:\Users\TJ\anaconda3\envs\ai_env2\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [64]:
# 성별만으로 생존여부를 판단하는 모델
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score

class MyTitanicCalssifier(BaseEstimator):
    def fit(self, X, y):
        # 아무것도 학습하지 않도록 오버라이드
        pass
    def predict(self, X):
        # 입력데이터의 성별컬럼(Sex)의 값에 따라 생존여부 예측
        # - 여성(0)인 경우 : 1 생존 예측
        # - 남성(1)인 경우 : 0 사망예측

        pred = np.zeros((X.shape[0], 1))
        # 모든 데이터를 순회하며 성별 검사
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i] == 0: # 여성인 경우
                    # 위치기반인덱싱
                    pred[i] = 1 # 생존
        return pred

    def score(self, X, y):
        # 정확도 측정
        return accuracy_score(y, self.predict(X))


my_clf = MyTitanicCalssifier()
my_clf.fit(X_train, y_train)

# 평가
print('학습셋 : ', my_clf.score(X_train, y_train))
print('평가셋 : ', my_clf.score(X_test, y_test))


학습셋 :  0.7893258426966292
평가셋 :  0.776536312849162


In [65]:
evaluate_binary_classification(y_test, my_clf.predict(X_test))

혼동행렬: 
 [[94 16]
 [24 45]]
정확도(Accuracy) :  0.78
정밀도(Precision) :  0.74
재현율(Recall) :  0.65
